Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_psi_t_var_formula">_psi_t_var_formula</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import a, b, c, k, m, n, t, P, defaults, Function
from proveit.logic import Equals, InSet
from proveit.numbers import (
        zero, one, two, i, e, pi, Add, Exp, Less, LessEq, Mult, Neg, subtract)
from proveit.numbers import Complex, Interval, Natural
from proveit.numbers.exponentiation import (
        add_one_right_in_exp, exp_eq_for_eq_base_and_exp, exponential_monotonocity)
from proveit.numbers.number_sets.natural_numbers import fold_forall_natural_pos
from proveit.physics.quantum.QPE import _phase, _phase_is_real

In [ ]:
%proving _psi_t_var_formula

In [ ]:
defaults.assumptions = _psi_t_var_formula.all_conditions()

In [ ]:
# the induction theorem for positive naturals
fold_forall_natural_pos

In [ ]:
# instantiate the induction theorem
induction_inst = fold_forall_natural_pos.instantiate(
    {Function(P,t):_psi_t_var_formula.instance_expr, m:t, n:t})

### Some Related Properties and Definitions Needed for Later Processing
Mainly: some domains and orderings. Notice that throughout this section devoted to the induction proof, $t$ is a _variable_, not a literal.

In [ ]:
# used when processing products involving the phase phi
_phase_is_real

In [ ]:
two_pow_t_var_less_one = subtract(Exp(two, t), one)

In [ ]:
two_pow_t_var_less_one.deduce_in_number_set(Natural)

In [ ]:
LessEq(zero, two_pow_t_var_less_one).prove()

In [ ]:
Less(t, Add(t, one)).prove()

In [ ]:
exponential_monotonocity

In [ ]:
exponential_monotonocity_inst = exponential_monotonocity.instantiate({a: two, b: t, c: Add(t, one)})

In [ ]:
# Used to allow a splitting of a summation into the sum of two summations
exponential_monotonocity_inst.derive_shifted(Neg(one))

For later summation index manipulations, we want to establish that $2^{t+1}-2^{t} = 2^{t}$ (and more specifically we will need $2^{t+1}-2^{t}-1 = 2^{t}-1$).

In [ ]:
add_one_right_in_exp

In [ ]:
two_to_quant_t_var_plus_1_factored = add_one_right_in_exp.instantiate(
        {a: two, b: t}).derive_reversed()

In [ ]:
index_shift_simplification = two_to_quant_t_var_plus_1_factored.substitution(
        subtract(Exp(two, Add(t, one)), Exp(two, t)))

In [ ]:
# index_shift_simplification = index_shift_simplification.inner_expr().rhs.simplify()

In [ ]:
index_shift_simplification = index_shift_simplification.right_add_both_sides(Neg(one))

In [ ]:
index_shift_simplification = index_shift_simplification.inner_expr().lhs.commute(init_idx=1, final_idx=2)

### Base Case

In [ ]:
base_case = induction_inst.antecedent.operands[0]

We have $|\psi_{t}\rangle$ defined as a tensor product (the result of the first phase of the quantum circuit, and the LHS of Nielsen & Chuang's identity 5.20 on pg 222):

In [ ]:
from proveit.physics.quantum.QPE import _psi_t_var_as_tensor_prod
_psi_t_var_as_tensor_prod

For $\psi'_{1}$, we prove a useful equality then instantiate the `psi_t_var_as_tensor_prod` with $t=1$:

In [ ]:
psi_1_def = _psi_t_var_as_tensor_prod.instantiate({t:one})

Then show that the summation formula also gives the same qbit result

In [ ]:
sum_0_to_1 = base_case.rhs

In [ ]:
sum_0_to_1_processed_01 = sum_0_to_1.inner_expr().operands[1].partitioning_first()

In [ ]:
# finish off the Base Case
base_case_jdgmt = sum_0_to_1_processed_01.sub_left_side_into(psi_1_def)

### Inductive Step

In [ ]:
inductive_step = induction_inst.antecedent.operands[1]

In [ ]:
defaults.assumptions = defaults.assumptions + inductive_step.conditions.entries

In [ ]:
scalar_coeff = inductive_step.instance_expr.rhs.scalar

First, partition the summation:
$\sum_{k=0}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} = \sum_{k=0}^{2^{t}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} + \sum_{k=2^{t}}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1}$

In [ ]:
summation_partition_01 = (
    inductive_step.instance_expr.rhs.operands[1]
    .partitioning(two_pow_t_var_less_one))

Then shift the second summation of that partition, so that the two summations then have the same index domain:

In [ ]:
summation_partition_02 = summation_partition_01.inner_expr().rhs.operands[1].shift(Neg(Exp(two, t)))

In [ ]:
# recall the following simplification engineered earlier in the nb:
index_shift_simplification

In [ ]:
summation_partition_03 = summation_partition_02.inner_expr().rhs.operands[1].domain.upper_bound.substitute(index_shift_simplification)

We want to rewrite the summand of that 2nd summation on the rhs now by (1) expanding the exponential term and (2) rewriting the $|k+2^t{\rangle}_{t+1}$ ket as $|1\rangle \otimes |k{\rangle}_t$. This takes a little work.

In [ ]:
rhs_2nd_sum = summation_partition_03.rhs.operands[1]

In [ ]:
summand_processed_01 = rhs_2nd_sum.summand.inner_expr().operands[0].exponent.distribution(
    4, assumptions=[*defaults.assumptions,
                    InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summand_processed_02 = summand_processed_01.inner_expr().rhs.operands[0].exponent_separate(assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# commute the NumKet num operands to format it for replacement later using the prepend_num_ket_with_one_ket thm
summand_processed_03 = summand_processed_02.inner_expr().rhs.operands[1].num.commute(
    assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
from proveit.physics.quantum.algebra import prepend_num_ket_with_one_ket
prepend_num_ket_with_one_ket

In [ ]:
prepend_num_ket_with_one_ket_inst = prepend_num_ket_with_one_ket.instantiate(
        {n: t, k: k}, assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summand_processed_04 = (
    summand_processed_03.inner_expr().rhs.operands[1]
    .substitute(prepend_num_ket_with_one_ket_inst))

In [ ]:
summand_processed_05 = summand_processed_04.inner_expr().rhs.operands[0].commute(assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# reminder of summation_partition_03
summation_partition_03

In [ ]:
# Now substitute our summand processing into the larger summations expression
summation_partition_04 = (
    summation_partition_03.inner_expr().rhs.operands[1].summand.substitute(
        summand_processed_05))

Also process the summand in the 1st sum on the rhs, converting the ket $|k\rangle_{t+1}$ to the tensor product $|0\rangle \otimes |k\rangle_{t}$:

In [ ]:
# pull up the relevant NumKet theorem
from proveit.physics.quantum.algebra import prepend_num_ket_with_zero_ket
prepend_num_ket_with_zero_ket

In [ ]:
# instantiate the theorem for our specific case
prepend_num_ket_with_zero_ket_inst = prepend_num_ket_with_zero_ket.instantiate(
        {n: t, k: k}, assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# use our instantiation to substitute
summation_partition_05 = (
    summation_partition_04.inner_expr().rhs.operands[0].summand.scaled.substitute(
    prepend_num_ket_with_zero_ket_inst))

We also then want to:

(1) pull the tensor product out of the 2nd summation, and

(2) pull the non-$k$-dependent exponential factor out of the 2nd summation.

In [ ]:
# for convenience:
temp_factors = summation_partition_04.rhs.operands[1].summand.operands[0]

In [ ]:
# for convenience:
temp_factor_01 = temp_factors.operands[0]

In [ ]:
# for convenience:
temp_factor_02 = temp_factors.operands[1]

In [ ]:
type(temp_factor_02.exponent.operands[2])

In [ ]:
# another convenience: an expr for the current domain of index k
k_domain = summation_partition_04.rhs.operands[1].conditions[0]

In [ ]:
from proveit.linear_algebra import VecSpaces
from proveit.numbers import Real
VecSpaces.default_field=Complex
summation_partition_06 = summation_partition_05.inner_expr().rhs.operands[1].factors_extract(
    field=None)

In [ ]:
summation_partition_07 = (
    summation_partition_06.inner_expr().rhs.operands[0].factors_extract())

Now factor out the common vector summation factor in the two terms of the rhs. We do this via the direct application of a manually-instantiated theorem about distribution of tensor product over a vector sum.

In [ ]:
from proveit.linear_algebra.tensors import tensor_prod_distribution_over_add
tensor_prod_distribution_over_add

In [ ]:
from proveit.linear_algebra import ScalarMult, TensorProd, VecAdd, VecSpaces
from proveit.physics.quantum import Ket
desired_expr = TensorProd(VecAdd(Ket(zero),  ScalarMult( temp_factor_01, Ket(one))),
                         summation_partition_07.rhs.operands[0].operands[1])

In [ ]:
vec_space_membership = desired_expr.deduce_in_vec_space(field=None)

In [ ]:
from proveit import ExprTuple
_V_sub, _K_sub, _a_sub, _i_sub, _b_sub, _j_sub, _c_sub, _k_sub  = (
    vec_space_membership.domain, VecSpaces.known_field(vec_space_membership.domain),
    (), zero,
    desired_expr.operands[0].operands, two, 
    ExprTuple(desired_expr.operands[1]), one)

In [ ]:
from proveit import a, b, c, j, k, K, V, Variable
i_var = Variable('i')
impl = tensor_prod_distribution_over_add.instantiate(
    {V: _V_sub, K: _K_sub, a: _a_sub, b: _b_sub, c: _c_sub,
     i_var: _i_sub, j: _j_sub, k: _k_sub})

In [ ]:
factored_sub = impl.derive_consequent().derive_reversed()

In [ ]:
summation_partition_09 = summation_partition_07.inner_expr().rhs.substitute(factored_sub)

Reintroduce the scalar coefficient:

In [ ]:
summation_partition_10 = inductive_step.instance_expr.rhs.inner_expr().scaled.substitution(summation_partition_09)

Manipulate our scalar coefficient in preparation for distribution a portion of it across the tensor product:

In [ ]:
# recall
scalar_coeff

In [ ]:
scalar_coeff_02 = scalar_coeff.inner_expr().denominator.exponent.distribution()

In [ ]:
scalar_coeff_03 = scalar_coeff_02.inner_expr().rhs.denominator.exponent_separate()

In [ ]:
from proveit.numbers import frac
scalar_coeff_04 = scalar_coeff_03.inner_expr().rhs.factor(frac(one, Exp(two, frac(one, two))))

In [ ]:
summation_partition_11 = summation_partition_10.inner_expr().rhs.scalar.substitute(scalar_coeff_04)

In [ ]:
# represent the desired (distributed scalar-product) outcome:
desired_expr = (
    TensorProd(
        ScalarMult(frac(one, Exp(two, frac(one, two))), summation_partition_11.rhs.scaled.operands[0]),
        ScalarMult(frac(one, Exp(two, frac(t, two))), summation_partition_11.rhs.scaled.operands[1])
    ))

In [ ]:
# let Prove-It prove the equality:
distrib_equality = Equals(summation_partition_11.rhs, desired_expr).prove()

In [ ]:
# now use that equality in our earlier work:
conclusion_01 = distrib_equality.sub_right_side_into(summation_partition_11)

In [ ]:
# Recall our inductive hypothesis:
for item in defaults.assumptions:
    if isinstance(item, Equals):
        inductive_hypothesis = item
inductive_hypothesis

In [ ]:
# use the inductive hypothesis:
conclusion_02 = inductive_hypothesis.sub_left_side_into(conclusion_01)

The RHS of conclusion_02 above should be equal to $|\psi_{t+1}\rangle$. Let's see how to get there. Ideally we could use the next three cells to show the rhs of the equality in `conclusion_2` is equal to $|\psi_{t+1}\rangle$

In [ ]:
_psi_t_var_as_tensor_prod

In [ ]:
psi_t_as_tensor_prod = _psi_t_var_as_tensor_prod.instantiate()

In [ ]:
psi_t_plus_one_as_tensor_prod = _psi_t_var_as_tensor_prod.instantiate({t:Add(t, one)})

In [ ]:
conclusion_02.inner_expr().rhs.operands[1].substitution(
        psi_t_as_tensor_prod)

The following three cell/steps needed so that the `substitute` step will be able to perform the desired simplification, pulling the coefficients out to the front:

In [ ]:
from proveit.numbers import greater_eq, NaturalPos
t_greater_eq_one = greater_eq(t, one).prove()

In [ ]:
t_minus_one_greater_eq_zero = t_greater_eq_one.right_add_both_sides(Neg(one))

In [ ]:
t_minus_one_greater_eq_zero.derive_negated()

Now this `substitute()` step will also manage to pull the scalar coefficients out to the front in the process:

In [ ]:
conclusion_03 = conclusion_02.inner_expr().rhs.operands[1].substitute(
        psi_t_as_tensor_prod)

In [ ]:
# conclusion_03.rhs.scaled.deduce_in_vec_space(field=Complex)

Now we partition the tensor product expression of $\psi_{t+1}$, so we can eventually show that it is equal to the rhs of conclusion_03.

In [ ]:
# to be used as a replacement in the partition step that follows
neg_t_plus_one_sub = Neg(Add(t, Neg(one))).distribution().derive_reversed()

In [ ]:
# we can partition the operands in the psi_{t+1} tensor prod expression
expr_range_partitioned = psi_t_plus_one_as_tensor_prod.rhs.scaled.operands[0].partition(
        Neg(t), replacements=[neg_t_plus_one_sub])

In [ ]:
# needed explicitly(somewhat oddly) for the next step of substitution
Add(t, one).commutation()

In [ ]:
psi_t_plus_one_as_tensor_prod_02 = psi_t_plus_one_as_tensor_prod.inner_expr().rhs.scaled.operands.substitute(
        expr_range_partitioned)

In [ ]:
psi_t_plus_one_as_tensor_prod_03 = psi_t_plus_one_as_tensor_prod_02.inner_expr().rhs.scalar.substitute(scalar_coeff_04)

In [ ]:
conclusion_03.inner_expr().rhs.substitute(psi_t_plus_one_as_tensor_prod_03.derive_reversed())

In [ ]:
# recall our earlier instantiation of the induction theorem:
induction_inst

We have proven both pieces of the antecedent (_i.e._, the base case and inductive case) of the instantiated induction theorem `induction_inst`, so we can now derive the consequent:

In [ ]:
induction_inst.derive_consequent()

In [ ]:
%qed